In [ ]:
# Setup embedding models and our vector database

from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()

openai_embeddings = OpenAIEmbeddings()

hf_embeddings = HuggingFaceEmbeddings()

openai_vector_store = FAISS.load_local("./data/nebraska-code", openai_embeddings, "openai", allow_dangerous_deserialization=True)
hf_vector_store = FAISS.load_local("./data/nebraska-code", hf_embeddings, "huggingface", allow_dangerous_deserialization=True)

In [ ]:
# Create our LLMs
import os
from langchain_openai import ChatOpenAI  
from langchain_huggingface import HuggingFaceEndpoint
from langchain import PromptTemplate


openai_llm = ChatOpenAI(  
  model="gpt-3.5-turbo",  
  temperature=0.7
)

hf_llm = HuggingFaceEndpoint(
  repo_id="mistralai/Mistral-7B-Instruct-v0.2", 
  temperature=0.7,
  huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
)

In [ ]:
# Use a local LLM
from dotenv import load_dotenv
from langchain_huggingface.llms import HuggingFacePipeline

load_dotenv()

local_llm = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    pipeline_kwargs={
        "temperature": 0.7,
    },
)

In [ ]:
# Create the prompt template

template = """
You are a helpful assistant helping attendees at Nebraska.Code(), a software development conference, make the most of their conference by answering questions about sessions and helping them find the best sessions to attend based on their interests. Your answers should be conversational - do not simply respond with the verbatim session data, but do make sure to include relevant information such as the room and times of the sessions. If there is more than one session that matches their query, feel free to reference them all. A bulleted list is acceptable in the output.
Here are some sessions that appear relevant to the user's question. Use these as you see fit to help answer the user's question:

{context}

Question: {question}
Answer: 

"""

prompt = PromptTemplate(
  template=template, 
  input_variables=["context", "question"]
)

In [ ]:
# Build the chains

from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

openai = (
  {"context": openai_vector_store.as_retriever(search_kwargs={'k': 10}),  "question": RunnablePassthrough()} 
  | prompt 
  | openai_llm
  | StrOutputParser() 
)

hf = (
  {"context": hf_vector_store.as_retriever(search_kwargs={'k': 10}),  "question": RunnablePassthrough()} 
  | prompt 
  | hf_llm
  | StrOutputParser() 
)

In [ ]:
print(openai.invoke("It's time to start leaning AI. Which sessions should I attend?"))

In [ ]:
print(hf.invoke("It's time to start leaning AI. Which sessions should I attend?"))